In [ ]:
import ray
from ray.util import state

In [ ]:
ray.init()

In [ ]:
state.list_tasks(filters=[("name", "=", "AnnotationImporter.import_from_tsv_row")], detail=True)[0]

In [ ]:
import time

# Define a Ray actor with a function
@ray.remote
class MyActor:
    def compute(self, x):
        return x * x

# Define a Ray remote function that calls a bunch of actors' functions
@ray.remote
def call_actors(actor_handles, values):
    # Call the 'compute' method on each actor with the corresponding value
    futures = [actor.compute.remote(val) for actor, val in zip(actor_handles, values)]
    return ray.get(futures)


In [ ]:

# Call the remote function and store the handle
actors = [MyActor.remote() for _ in range(3)]
call_handle = call_actors.remote(actors, [1, 2, 3])


In [ ]:


# Get the task id of the remote function
task = state.get_task(call_handle)
if task:
    parent_task_id = task.task_id
    # Use parent_task_id to get child tasks
    child_tasks = state.list_tasks(filters=[("parent_task_id", "=", parent_task_id)], detail=True)
    for t in child_tasks:
        print(f"Child task {t.name} (id={t.task_id}) state: {t.state}")
else:
    print("No call_actors tasks found.")
